In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Any results you write to the current directory are saved as output.

In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import itertools
import sys
from pandas import DataFrame
from keras.models import Sequential

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
%matplotlib inline
#from imutils import paths
import argparse
import random
import pickle
import cv2

from bs4 import BeautifulSoup 
import re # For regular expressions


# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


#For Tokenizing and pad sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



import string

#For Lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

In [ ]:
import pandas as pd
text_emotion = pd.read_csv("../input/text_emotion.csv")
text_emotion=text_emotion[text_emotion.sentiment != 'enthusiasm']
text_emotion=text_emotion[text_emotion.sentiment != 'neutral']
text_emotion=text_emotion[text_emotion.sentiment != 'empty']
text_emotion=text_emotion[text_emotion.sentiment != 'surprise']
text_emotion=text_emotion[text_emotion.sentiment != 'boredom']
text_emotion=text_emotion[text_emotion.sentiment != 'relief']
text_emotion=text_emotion[text_emotion.sentiment != 'worry']
text_emotion=text_emotion[text_emotion.sentiment != 'love']
text_emotion=text_emotion[text_emotion.sentiment != 'fun']
text_emotion['sentiment'] = text_emotion['sentiment'].map({'sadness': "sadness",'hate':'anger',
                                                           'happiness': "joy",
                                                           'anger': "anger"})
text_emotion=text_emotion.drop(columns=['tweet_id', 'author'])

text_emotion = text_emotion.reindex(sorted(text_emotion.columns), axis=1)
text_emotion.columns = ['content', 'Affect']

In [ ]:
import pandas as pd
test = pd.read_csv("../input/DS-Test.csv",encoding='ISO-8859-1')
train = pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')
train = train[train.Intensity != '0']
train = train[train.Intensity != 0]
train=train.drop(columns=['ID','Intensity'])
train.columns=['content','Affect']
train=train.append(text_emotion)
train1=pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')

#train1 = train1[train1.Intensity != '0']
train1 = train1[train1.Intensity != 0]

In [ ]:
train1

In [ ]:
trainsad = pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')
trainfear = pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')
trainanger = pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')
trainjoy = pd.read_csv("../input/DS-Training.csv",encoding='ISO-8859-1')
trainsad=trainsad[trainsad.Affect == "sadness"]
trainjoy=trainjoy[trainjoy.Affect == "joy"]
trainfear=trainfear[trainfear.Affect == "fear"]
trainanger=trainanger[trainanger.Affect == "anger"]



trainsad=trainsad[trainsad.Intensity != 0]
trainjoy=trainjoy[trainjoy.Intensity != 0]
trainfear=trainfear[trainfear.Intensity != 0]
trainanger=trainanger[trainanger.Intensity != 0]

In [ ]:
trainfear

In [ ]:
# This function converts a text to a sequence of words.
def review_wordlist(review, remove_stopwords=False):
    # Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # Converting to lower case and splitting
    trains_content = review_text.lower().split()
    # Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        trains_content = [w for w in trains_content if not w in stops]

    
    return(trains_content)


# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
#nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # This returns the list of lists
    return sentences




#Removing punctuation problems
train["content"] = train["content"].str.replace('[^\w\s]','')
train["content"].head()

#Removing common words
freq = pd.Series(' '.join(train["content"]).split()).value_counts()[:10]
freq = list(freq.index)
train["content"] = train["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(train["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
train["content"] = train["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
train["content"] = train["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (train["content"])

sentences = []
print("Parsing sentences from training set")
for review in train["content"]:
    sentences += review_sentences(review, tokenizer)
    
    
    
    
    
    
#for intensity





#Removing punctuation problems
train1["content"] = train1["content"].str.replace('[^\w\s]','')
train1["content"].head()

#Removing common words
freq1 = pd.Series(' '.join(train1["content"]).split()).value_counts()[:10]
freq1 = list(freq1.index)
train1["content"] = train1["content"].apply(lambda x1: " ".join(x1 for x1 in x1.split() if x1 not in freq1))
train1["content"].head()

#Removing rarely occuring words
freq1 = pd.Series(' '.join(train1["content"]).split()).value_counts()[-10:]
freq1 = list(freq1.index)
train1["content"] = train1["content"].apply(lambda x1: " ".join(x1 for x1 in x1.split() if x1 not in freq))
train1["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
train1["content"] = train1["content"].apply(lambda x1: " ".join([Word(word).lemmatize() for word in x1.split()]))
print (train1["content"])

sentences1 = []
print("Parsing sentences from training set")
for review in train1["content"]:
    sentences1 += review_sentences(review, tokenizer)
    
    
    
    
#sad



#Removing punctuation problems
trainsad["content"] = trainsad["content"].str.replace('[^\w\s]','')
trainsad["content"].head()

#Removing common words
freq = pd.Series(' '.join(trainsad["content"]).split()).value_counts()[:10]
freq = list(freq.index)
trainsad["content"] = trainsad["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainsad["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(trainsad["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
trainsad["content"] = trainsad["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainsad["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
trainsad["content"] = trainsad["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (trainsad["content"])

sentences = []
print("Parsing sentences from training set")
for review in trainsad["content"]:
    sentences += review_sentences(review, tokenizer)

    
    
    
#joy





#Removing punctuation problems
trainjoy["content"] = trainjoy["content"].str.replace('[^\w\s]','')
trainjoy["content"].head()

#Removing common words
freq = pd.Series(' '.join(trainjoy["content"]).split()).value_counts()[:10]
freq = list(freq.index)
trainjoy["content"] = trainjoy["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainjoy["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(trainjoy["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
trainjoy["content"] = trainjoy["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainjoy["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
trainjoy["content"] = trainjoy["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (trainjoy["content"])

sentences = []
print("Parsing sentences from training set")
for review in trainjoy["content"]:
    sentences += review_sentences(review, tokenizer)
  
    
    
#fear





#Removing punctuation problems
trainfear["content"] = trainfear["content"].str.replace('[^\w\s]','')
trainfear["content"].head()

#Removing common words
freq = pd.Series(' '.join(trainfear["content"]).split()).value_counts()[:10]
freq = list(freq.index)
trainfear["content"] = trainfear["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainfear["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(trainfear["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
trainfear["content"] = trainfear["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainfear["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
trainfear["content"] = trainfear["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (trainfear["content"])

sentences = []
print("Parsing sentences from training set")
for review in trainfear["content"]:
    sentences += review_sentences(review, tokenizer)
  
    
    
    
#anger



#Removing punctuation problems
trainanger["content"] = trainanger["content"].str.replace('[^\w\s]','')
trainanger["content"].head()

#Removing common words
freq = pd.Series(' '.join(trainanger["content"]).split()).value_counts()[:10]
freq = list(freq.index)
trainanger["content"] = trainanger["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainanger["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(trainanger["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
trainanger["content"] = trainanger["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
trainanger["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
trainanger["content"] = trainanger["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (trainanger["content"])

sentences = []
print("Parsing sentences from training set")
for review in trainanger["content"]:
    sentences += review_sentences(review, tokenizer)
  
    
    
    
    
    
    
    
    
    
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)



#for intensity
# Creating the model and setting values for the various parameters
num_features1 = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model1 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model1.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name1 = "300features_40minwords_10context"
model1.save(model_name1)


# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [ ]:
#sad

num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model2 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model2.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name2 = "300features_40minwords_10context"
model2.save(model_name2)




#joy

num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model3 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model3.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name3 = "300features_40minwords_10context"
model3.save(model_name3)


#fear


num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model4 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model4.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name4 = "300features_40minwords_10context"
model4.save(model_name4)



#anger



num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model5 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model5.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name5 = "300features_40minwords_10context"
model5.save(model_name5)

In [ ]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in train['content']:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
#print (trainDataVecs)

# Calculating average feature vector for training set
clean_train_reviews1 = []
for review in train1['content']:
    clean_train_reviews1.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs1 = getAvgFeatureVecs(clean_train_reviews1, model1, num_features1)
#print (trainDataVecs)


# Calculating average feature vactors for test set     
clean_test_reviews = []
for review in test["Tweet"]:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
print (tokenizer)
tokenizer.fit_on_texts(train['content'].values)
X = tokenizer.texts_to_sequences(train["content"].values)
X = pad_sequences(X)
print (X.shape[1])


max_features = 2000
tokenizer1 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer1)
tokenizer1.fit_on_texts(train1['content'].values)
X1 = tokenizer1.texts_to_sequences(train1["content"].values)
X1 = pad_sequences(X1)
print (X1.shape[1])

In [ ]:
clean_train_reviews2 = []
for review in trainsad['content']:
    clean_train_reviews2.append(review_wordlist(review, remove_stopwords=True))
trainDataVecs2 = getAvgFeatureVecs(clean_train_reviews2, model2, num_features)


clean_train_reviews3 = []
for review in trainjoy['content']:
    clean_train_reviews3.append(review_wordlist(review, remove_stopwords=True))
trainDataVecs3 = getAvgFeatureVecs(clean_train_reviews3, model3, num_features)

clean_train_reviews4 = []
for review in trainfear['content']:
    clean_train_reviews4.append(review_wordlist(review, remove_stopwords=True))  
trainDataVecs4 = getAvgFeatureVecs(clean_train_reviews4, model4, num_features)


clean_train_reviews5 = []
for review in trainanger['content']:
    clean_train_reviews5.append(review_wordlist(review, remove_stopwords=True))
trainDataVecs5 = getAvgFeatureVecs(clean_train_reviews5, model5, num_features)




max_features = 2000
tokenizer2 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer2)
tokenizer2.fit_on_texts(trainsad['content'].values)
X2 = tokenizer2.texts_to_sequences(trainsad["content"].values)
X2 = pad_sequences(X2)
print (X2.shape[1])




max_features = 2000
tokenizer3 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer3)
tokenizer3.fit_on_texts(trainjoy['content'].values)
X3 = tokenizer3.texts_to_sequences(trainjoy["content"].values)
X3 = pad_sequences(X3)
print (X3.shape[1])



max_features = 2000
tokenizer4 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer4)
tokenizer4.fit_on_texts(trainfear['content'].values)
X4 = tokenizer4.texts_to_sequences(trainfear["content"].values)
X4 = pad_sequences(X4)
print (X4.shape[1])



max_features = 2000
tokenizer5 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer5)
tokenizer5.fit_on_texts(trainanger['content'].values)
X5 = tokenizer5.texts_to_sequences(trainanger["content"].values)
X5 = pad_sequences(X5)
print (X5.shape[1])

In [ ]:
embed_dim = 128
lstm_out = 256
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


embed_dim1 = 128
lstm_out1 = 256
model1 = Sequential()
model1.add(Embedding(max_features, embed_dim1,input_length = X1.shape[1]))
model1.add(SpatialDropout1D(0.4))
model1.add(LSTM(lstm_out1, dropout=0.3, recurrent_dropout=0.3))
model1.add(Dense(4,activation='softmax'))
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.summary())




embed_dim2 = 128
lstm_out2 = 256
model2 = Sequential()
model2.add(Embedding(max_features, embed_dim2,input_length = X2.shape[1]))
model2.add(SpatialDropout1D(0.4))
model2.add(LSTM(lstm_out2, dropout=0.3, recurrent_dropout=0.3))
model2.add(Dense(3,activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())




embed_dim3 = 128
lstm_out3 = 256
model3 = Sequential()
model3.add(Embedding(max_features, embed_dim3,input_length = X3.shape[1]))
model3.add(SpatialDropout1D(0.4))
model3.add(LSTM(lstm_out3, dropout=0.3, recurrent_dropout=0.3))
model3.add(Dense(3,activation='softmax'))
model3.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model3.summary())



embed_dim4 = 128
lstm_out4 = 256
model4 = Sequential()
model4.add(Embedding(max_features, embed_dim4,input_length = X4.shape[1]))
model4.add(SpatialDropout1D(0.4))
model4.add(LSTM(lstm_out4, dropout=0.3, recurrent_dropout=0.3))
model4.add(Dense(3,activation='softmax'))
model4.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model4.summary())






embed_dim5 = 128
lstm_out5 = 256
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim5,input_length = X5.shape[1]))
model5.add(SpatialDropout1D(0.4))
model5.add(LSTM(lstm_out5, dropout=0.3, recurrent_dropout=0.3))
model5.add(Dense(3,activation='softmax'))
model5.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model5.summary())








In [ ]:
Y = pd.get_dummies(train['Affect']).values #this
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

Y1 = pd.get_dummies(train1['Intensity']).values #this
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, test_size = 0.3, random_state = 42)

Y2 = pd.get_dummies(trainsad['Intensity']).values #this
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2,Y2, test_size = 0.3, random_state = 42)

Y3 = pd.get_dummies(trainjoy['Intensity']).values #this
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3,Y3, test_size = 0.3, random_state = 42)


Y4 = pd.get_dummies(trainfear['Intensity']).values #this
X4_train, X4_test, Y4_train, Y4_test = train_test_split(X4,Y4, test_size = 0.3, random_state = 42)

Y5 = pd.get_dummies(trainanger['Intensity']).values #this
X5_train, X5_test, Y5_train, Y5_test = train_test_split(X5,Y5, test_size = 0.3, random_state = 42)



In [ ]:
batch_size = 32
model2.fit(X2_train, Y2_train, epochs = 150, batch_size=batch_size, verbose = 2)

In [ ]:
batch_size = 32
model3.fit(X3_train, Y3_train, epochs = 150, batch_size=batch_size, verbose = 2)

In [ ]:
batch_size = 32
model4.fit(X4_train, Y4_train, epochs = 150, batch_size=batch_size, verbose = 2)

In [ ]:
batch_size = 32
model5.fit(X5_train, Y5_train, epochs = 150, batch_size=batch_size, verbose = 2)

In [ ]:
psad=model2.predict(X2_test)
b = np.zeros_like(psad)
b[np.arange(len(psad)), psad.argmax(1)] = 1
b=b.astype(int)
c = np.zeros_like(Y2_test)
c[np.arange(len(Y2_test)), Y2_test.argmax(1)] = 1
c=c.astype(int)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(c,b))


pjoy=model3.predict(X3_test)
b = np.zeros_like(pjoy)
b[np.arange(len(pjoy)), pjoy.argmax(1)] = 1
b=b.astype(int)
c = np.zeros_like(Y3_test)
c[np.arange(len(Y3_test)), Y3_test.argmax(1)] = 1
c=c.astype(int)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(c,b))



pfear=model4.predict(X4_test)
b = np.zeros_like(pfear)
b[np.arange(len(pfear)), pfear.argmax(1)] = 1
b=b.astype(int)
c = np.zeros_like(Y4_test)
c[np.arange(len(Y4_test)), Y4_test.argmax(1)] = 1
c=c.astype(int)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(c,b))



panger=model5.predict(X5_test)
b = np.zeros_like(panger)
b[np.arange(len(panger)), panger.argmax(1)] = 1
b=b.astype(int)
c = np.zeros_like(Y5_test)
c[np.arange(len(Y5_test)), Y5_test.argmax(1)] = 1
c=c.astype(int)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(c,b))

In [ ]:
#Decision tree classifier on both emotion and intenisty


from sklearn.tree import tree
dtree = tree.DecisionTreeClassifier()
dtree2 = tree.DecisionTreeClassifier()
dtree.fit(X_train, Y_train)
dtree2.fit(X1_train, Y1_train)
predictions = dtree.predict(X_test)
predictions2 = dtree2.predict(X1_test)
from sklearn.metrics import classification_report,confusion_matrix
print ("F1 Score of emotion classification from Decision Tree Classifier")
print (" ")
print(classification_report(Y_test,predictions))
print (" ")
print ("F1 Score of intensity classification from Decision Tree Classifier")
print (" ")
print(classification_report(Y1_test,predictions2))

In [ ]:
from sklearn.tree import tree
dtree3 = tree.DecisionTreeClassifier()
dtree3.fit(X2_train, Y2_train)
p = dtree3.predict(X2_test)
from sklearn.metrics import classification_report,confusion_matrix
print ("F1 Score of emotion classification from Decision Tree Classifier")
print (" ")
print(classification_report(Y2_test,p))

In [ ]:
#Decision Tree Regressor for intensity
print ("Decision tree regressor F1 score for intenisty")
print ("")
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state = 0)  
regressor.fit(X1_train, Y1_train) 
predictions = regressor.predict(X1_test)
predictions = predictions.astype(int)
print(classification_report(Y1_test,predictions))

In [ ]:
#Rndom Forest classifier for both emotion and intensity
from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier()
rforest.fit(X_train, Y_train)
rforest1 = RandomForestClassifier()
rforest1.fit(X1_train, Y1_train)
p = rforest.predict(X_test)
print ("F1 score for Emotion Classification from Random Forest Classifier")
print ("")
print(classification_report(Y_test,p))
p1 = rforest1.predict(X1_test)
print ("F1 score for Intensity Classification from Random Forest Classifier")
print ("")
print(classification_report(Y1_test,p1))

In [ ]:
# Random Forest Regressor for intenisty
from sklearn.ensemble import RandomForestRegressor
rfregressor = RandomForestRegressor(n_estimators = 200
                                    , random_state = 1) 
rfregressor.fit(X1_train, Y1_train) 
rfpredictions = rfregressor.predict(X1_test)
b = np.zeros_like(rfpredictions)
b[np.arange(len(rfpredictions)), rfpredictions.argmax(1)] = 1
b=b.astype(int)
rfpredictions=b
#rfpredictions = rfpredictions.astype(int)
print (" F1 score for intenisty using random forest regressor")
print ("")
print(classification_report(Y1_test,rfpredictions))

In [ ]:
#naive bayes 

Y=[]
x=0
l=len(Y1_train)
val=0
while x<l:
    val=np.where(Y1_train[x] == 1)
    Y.append(val[0][0])
    x+=1
from sklearn.naive_bayes import GaussianNB
mnb = GaussianNB()
mnb.fit(X1_train, Y)
y_pred = mnb.predict(X1_test)
Y1=[]
x=0
l=len(Y1_test)
val=0
while x<l:
    val=np.where(Y1_test[x] == 1)
    Y1.append(val[0][0])
    x+=1
    from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y1, y_pred))

In [ ]:
#naive bayes emotion

Y=[]
x=0
l=len(Y_train)
val=0
while x<l:
    val=np.where(Y_train[x] == 1)
    Y.append(val[0][0])
    x+=1
from sklearn.naive_bayes import GaussianNB
mnb = GaussianNB()
mnb.fit(X_train, Y)
y_pred = mnb.predict(X_test)
Y1=[]
x=0
l=len(Y_test)
val=0
while x<l:
    val=np.where(Y_test[x] == 1)
    Y1.append(val[0][0])
    x+=1
    from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y1, y_pred))

In [ ]:
#linear Regression 

from sklearn import datasets, linear_model, metrics 
reg = linear_model.LinearRegression() 

reg.fit(X1_train, Y1_train) 
p=reg.predict(X1_test)
b = np.zeros_like(p)
b[np.arange(len(p)), p.argmax(1)] = 1
b=b.astype(int)
p=b
print(classification_report(Y1_test,p))

In [ ]:
#SVM 

Y=[]
x=0
l=len(Y1_train)
val=0
while x<l:
    val=np.where(Y1_train[x] == 1)
    Y.append(val[0][0])
    x+=1
from sklearn import svm
clf = svm.SVC(max_iter=1000000)
clf.fit(X1_train, Y)
p1=clf.predict(X1_test)
Y1=[]
x=0
l=len(Y1_test)
val=0
while x<l:
    val=np.where(Y1_test[x] == 1)
    Y1.append(val[0][0])
    x+=1
print(classification_report(Y1,p1))

In [ ]:

Y=[]
x=0
l=len(Y_train)
val=0
while x<l:
    val=np.where(Y_train[x] == 1)
    Y.append(val[0][0])
    x+=1
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, Y)
p1=clf.predict(X_test)
Y1=[]
x=0
l=len(Y_test)
val=0
while x<l:
    val=np.where(Y_test[x] == 1)
    Y1.append(val[0][0])
    x+=1
print(classification_report(Y1,p1))

In [ ]:
#Logistic Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score
Y=[]
x=0
l=len(Y_train)
val=0
while x<l:
    val=np.where(Y_train[x] == 1)
    Y.append(val[0][0])
    x+=1
logmodel = LogisticRegression()
logmodel.fit(X_train, Y)
p=logmodel.predict(X_test)
Y1=[]
x=0
l=len(Y_test)
val=0
while x<l:
    val=np.where(Y_test[x] == 1)
    Y1.append(val[0][0])
    x+=1
print(classification_report(Y1,p))

In [ ]:
from sklearn import datasets, linear_model, metrics 
reg = linear_model.LinearRegression() 

reg.fit(X_train, Y_train) 
p=reg.predict(X_test)
b = np.zeros_like(p)
b[np.arange(len(p)), p.argmax(1)] = 1
b=b.astype(int)
p=b
print(classification_report(Y_test,p))

In [ ]:
#emotion

objects = ('Neural \n Networks', 'Random \n Forests', 'Decision \n Trees','SVM', 'Logistic \n Regression')
y_pos = np.arange(len(objects))
performance = [0.74,0.4,0.5,0.4,0.41]


plt.bar(y_pos, performance,align='center', alpha=1)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy Scores')
plt.title('Emotion Classification accuracies from different models')
plt.show()

In [ ]:
#Intensity

objects = ('Neural \n Networks', 'Random \n Forests', 'Decision \n Trees','SVM', 'Logistic \n Regression','Naive \n Bayes')
y_pos = np.arange(len(objects))
performance = [0.53,0.36,0.37,0.36,0.36,0.37]


plt.bar(y_pos, performance,align='center', alpha=1)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy Scores')
plt.title('Intensity Classification accuracies from different models')
plt.show()